<a href="https://colab.research.google.com/github/jessyy2006/MachineLearningProjects/blob/main/Predicting_Gini_w_Poly_Regression.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.preprocessing import PolynomialFeatures
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.linear_model import Lasso

## Mount Google Drive

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


https://saturncloud.io/blog/how-to-import-files-from-google-drive-to-colab/

# Import/Read Data

https://docs.google.com/spreadsheets/d/1Nk679xoygjlkTVKb-sfRY2d-xKRF_t3KtQVu9838W28/edit?gid=0#gid=0

In [ ]:
# grab data -- change path for your own file
df1 = pd.read_csv('/content/drive/MyDrive/Inequality Dataset Latent Variable Analysis - edited csv (trying to populate).csv')

# print first few rows -- could also use df.head()
print (df1.iloc[:3])
# and get summary stats on variables
print (df1.describe())
df1

       country  federalism_gt  id                     region_wb  gdp  \
0  Afghanistan            0.0 NaN  Middle East and North Africa  NaN   
1      Albania            0.0 NaN       Europe and Central Asia  NaN   
2      Algeria            0.0 NaN  Middle East and North Africa  NaN   

   statehiste1500_02n  origtime2  legor_so       eleva  avg_temp  ...  \
0            0.422657      40000         0  1836.79830      13.8  ...   
1            0.374967      45000         1   725.77765      12.5  ...   
2            0.405222      40000         0   585.31787      23.4  ...   

   v1_a_lum00pc  eur_pct_homog   mal_max  tech1500_ext  gini_disp_ext  \
0      0.732304      50.000000 -0.133697         0.617      31.700001   
1      0.037976      42.840736 -0.744265         0.825      38.900002   
2      0.892659      49.142891 -0.286520         0.783      35.500000   

   sd_index_ext    rugged       soil  tropical  adj_index  
0     -0.564311  2.518146  27.848703       0.0   1.405038  
1    

,country,federalism_gt,id,region_wb,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,...,v1_a_lum00pc,eur_pct_homog,mal_max,tech1500_ext,gini_disp_ext,sd_index_ext,rugged,soil,tropical,adj_index
0,Afghanistan,0.0,NaN,Middle East and North Africa,NaN,0.422657,40000,0,1836.798300,13.800000,...,0.732304,50.000000,-0.133697,0.617,31.700001,-0.564311,2.518146,27.848703,0.000000,1.405038
1,Albania,0.0,NaN,Europe and Central Asia,NaN,0.374967,45000,1,725.777650,12.500000,...,0.037976,42.840736,-0.744265,0.825,38.900002,-1.452140,3.427058,68.088409,0.000000,1.451480
2,Algeria,0.0,NaN,Middle East and North Africa,NaN,0.405222,40000,0,585.317870,23.400000,...,0.892659,49.142891,-0.286520,0.783,35.500000,-1.716895,0.509751,9.590019,0.000000,0.422037
3,Angola,0.0,NaN,Sub-Saharan Africa,NaN,0.050186,135000,0,1099.466300,21.600000,...,0.815747,48.803310,1.609371,0.167,49.799999,-0.101569,0.858216,26.676142,44.346245,-2.669100
4,Antigua,1.0,NaN,Latin America and the Caribbean,NaN,NaN,58917,0,60.476498,26.500000,...,0.000000,47.932915,NaN,0.133,47.099998,3.610510,0.006226,100.000000,100.000000,0.162782
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Venezuela,2.0,20431.0,Latin America and the Caribbean,19012.9920,0.000000,15000,0,374.298950,25.900000,...,0.438168,10.905635,-0.287601,0.133,39.200001,3.430511,0.633912,21.567352,96.751541,-0.089991
193,Vietnam,0.0,NaN,South Asia,NaN,0.474578,75000,1,420.906520,24.799999,...,0.569244,49.998993,0.939662,0.800,34.500000,-0.965899,2.053842,12.150098,56.534649,1.037454
194,Yemen,0.0,NaN,Middle East and North Africa,NaN,0.591734,85000,0,999.000000,24.700001,...,NaN,50.000000,1.517469,0.483,37.000000,-0.761840,2.323113,0.000000,0.000000,0.815853
195,Zambia,0.0,20778.0,Sub-Saharan Africa,2619.7214,0.000000,135000,0,1148.561600,21.700001,...,0.643952,49.318665,1.609371,0.192,55.200001,-0.363028,0.532584,23.114420,17.132454,-2.615809


## Edit DataFrame


In [ ]:
df2 = df1

# make geographical region variable into 8 dummy binary vars
temp_geog_vars = pd.get_dummies(df2['region_wb'], dtype = int)
temp_geog_vars

df2 = pd.concat([df2, temp_geog_vars], axis=1)
df2['avg_temp'].isna().sum()
df2['origtime2'].isna().sum()
df2['tropical'].isna().sum()

7

In [ ]:
df2.drop('region_wb', axis=1, inplace=True)

In [ ]:
df2

,country,federalism_gt,id,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,...,soil,tropical,adj_index,East Asia and the Pacific,Europe and Central Asia,Latin America and the Caribbean,Middle East and North Africa,North America,South Asia,Sub-Saharan Africa
0,Afghanistan,0.0,NaN,NaN,0.422657,40000,0,1836.798300,13.800000,0.769345,...,27.848703,0.000000,1.405038,0,0,0,1,0,0,0
1,Albania,0.0,NaN,NaN,0.374967,45000,1,725.777650,12.500000,0.220426,...,68.088409,0.000000,1.451480,0,1,0,0,0,0,0
2,Algeria,0.0,NaN,NaN,0.405222,40000,0,585.317870,23.400000,0.339400,...,9.590019,0.000000,0.422037,0,0,0,1,0,0,0
3,Angola,0.0,NaN,NaN,0.050186,135000,0,1099.466300,21.600000,0.786720,...,26.676142,44.346245,-2.669100,0,0,0,0,0,0,1
4,Antigua,1.0,NaN,NaN,NaN,58917,0,60.476498,26.500000,0.164256,...,100.000000,100.000000,0.162782,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Venezuela,2.0,20431.0,19012.9920,0.000000,15000,0,374.298950,25.900000,0.496600,...,21.567352,96.751541,-0.089991,0,0,1,0,0,0,0
193,Vietnam,0.0,NaN,NaN,0.474578,75000,1,420.906520,24.799999,0.238308,...,12.150098,56.534649,1.037454,0,0,0,0,0,1,0
194,Yemen,0.0,NaN,NaN,0.591734,85000,0,999.000000,24.700001,NaN,...,0.000000,0.000000,0.815853,0,0,0,1,0,0,0
195,Zambia,0.0,20778.0,2619.7214,0.000000,135000,0,1148.561600,21.700001,0.780800,...,23.114420,17.132454,-2.615809,0,0,0,0,0,0,1


### Fill in missing values for relevant columns

Fill conditionally based on region

In [ ]:
missing_values = df2.isnull().sum()

# Filter columns with missing values
columns_with_missing = missing_values[missing_values > 0]
columns_with_missing

,0
federalism_gt,2
id,128
gdp,128
statehiste1500_02n,39
eleva,9
avg_temp,5
al_ethnic,14
al_language,20
al_religion,11
english_legal_origin,10


In [ ]:
regions = [
    'Europe and Central Asia',
    'East Asia and the Pacific',
    'Latin America and the Caribbean',
    'Middle East and North Africa',
    'North America',
    'South Asia',
    'Sub-Saharan Africa'
]

columns_to_fill = ['avg_temp', 'tropical', 'origtime2', 'arablelandoflandareaag', 'pmean', 'pln_sxhr_mean', 'soil', 'irri_impact5',
                   'frstdays', 'gdp', 'rugged', 'sd_emeanclip', 'statehiste1500_02n', 'employmentinagricultureof', 'agyears_ext', 'eleva',
                   'al_ethnic', 'al_language', 'al_religion', 'eur_pct_est_smooth', 'eur_pct_homog', 'mal_max', 'tech1500_ext', 'sd_index_ext', 'adj_index', 'healthexpenditurepublicof',
    'maddison_gdppc_1990_estimate_ln',
    'lp_lat_abst_fill',
    'mountains',
    'log_ocdistance_new',
    'v2x_corr',
    'v2x_polyarchy',
    'urbanpopulationoftotalpop',
    'dist2suitable_km_new',
    'agric_value_added',
    'generalgovernmentfinalconsump_ln',
    'fixedtelephonesubscriptionsp',
    'accesstoelectricityofpopu',
    'populationdensitypeoplepers_ln',
    'capacity',
    'tradeofgdpnetrdgnfszs_ln',
    'lsc_ipo',
    'popd_1500ad',
    'popd_1500ad_ln',
    'grg_a_lum00pc',
    'v1_a_lum00pc',
    'english_legal_origin']

for region in regions:
    for column in columns_to_fill:
        df2.loc[(df2[region] == 1) & (df2[column].isna()), column] = df2.loc[df2[region] == 1, column].mean()

<ipython-input-12-605deb0bc619>:38: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '34301.73469387755' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  df2.loc[(df2[region] == 1) & (df2[column].isna()), column] = df2.loc[df2[region] == 1, column].mean()


Deleting cols with lots of missingness (>50 vals)

In [ ]:
df2 = df2.drop(columns = ['id', 'taxrevenueofgdpgctaxt_2', 'sd_statehist'])

In [ ]:
df2

,country,federalism_gt,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,al_language,...,soil,tropical,adj_index,East Asia and the Pacific,Europe and Central Asia,Latin America and the Caribbean,Middle East and North Africa,North America,South Asia,Sub-Saharan Africa
0,Afghanistan,0.0,28260.808800,0.422657,40000.0,0,1836.798300,13.800000,0.769345,0.614146,...,27.848703,0.000000,1.405038,0,0,0,1,0,0,0
1,Albania,0.0,30566.399796,0.374967,45000.0,1,725.777650,12.500000,0.220426,0.039925,...,68.088409,0.000000,1.451480,0,1,0,0,0,0,0
2,Algeria,0.0,28260.808800,0.405222,40000.0,0,585.317870,23.400000,0.339400,0.442662,...,9.590019,0.000000,0.422037,0,0,0,1,0,0,0
3,Angola,0.0,6747.803429,0.050186,135000.0,0,1099.466300,21.600000,0.786720,0.787019,...,26.676142,44.346245,-2.669100,0,0,0,0,0,0,1
4,Antigua,1.0,12679.182646,0.062178,58917.0,0,60.476498,26.500000,0.164256,0.106328,...,100.000000,100.000000,0.162782,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Venezuela,2.0,19012.992000,0.000000,15000.0,0,374.298950,25.900000,0.496600,0.068580,...,21.567352,96.751541,-0.089991,0,0,1,0,0,0,0
193,Vietnam,0.0,7304.192150,0.474578,75000.0,1,420.906520,24.799999,0.238308,0.237708,...,12.150098,56.534649,1.037454,0,0,0,0,0,1,0
194,Yemen,0.0,28260.808800,0.591734,85000.0,0,999.000000,24.700001,0.450171,0.007982,...,0.000000,0.000000,0.815853,0,0,0,1,0,0,0
195,Zambia,0.0,2619.721400,0.000000,135000.0,0,1148.561600,21.700001,0.780800,0.873408,...,23.114420,17.132454,-2.615809,0,0,0,0,0,0,1


In [ ]:
# finally, drop rows that have a missing value
df2.dropna(inplace=True)
df2

,country,federalism_gt,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,al_language,...,soil,tropical,adj_index,East Asia and the Pacific,Europe and Central Asia,Latin America and the Caribbean,Middle East and North Africa,North America,South Asia,Sub-Saharan Africa
0,Afghanistan,0.0,28260.808800,0.422657,40000.0,0,1836.798300,13.800000,0.769345,0.614146,...,27.848703,0.000000,1.405038,0,0,0,1,0,0,0
1,Albania,0.0,30566.399796,0.374967,45000.0,1,725.777650,12.500000,0.220426,0.039925,...,68.088409,0.000000,1.451480,0,1,0,0,0,0,0
2,Algeria,0.0,28260.808800,0.405222,40000.0,0,585.317870,23.400000,0.339400,0.442662,...,9.590019,0.000000,0.422037,0,0,0,1,0,0,0
3,Angola,0.0,6747.803429,0.050186,135000.0,0,1099.466300,21.600000,0.786720,0.787019,...,26.676142,44.346245,-2.669100,0,0,0,0,0,0,1
4,Antigua,1.0,12679.182646,0.062178,58917.0,0,60.476498,26.500000,0.164256,0.106328,...,100.000000,100.000000,0.162782,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Venezuela,2.0,19012.992000,0.000000,15000.0,0,374.298950,25.900000,0.496600,0.068580,...,21.567352,96.751541,-0.089991,0,0,1,0,0,0,0
193,Vietnam,0.0,7304.192150,0.474578,75000.0,1,420.906520,24.799999,0.238308,0.237708,...,12.150098,56.534649,1.037454,0,0,0,0,0,1,0
194,Yemen,0.0,28260.808800,0.591734,85000.0,0,999.000000,24.700001,0.450171,0.007982,...,0.000000,0.000000,0.815853,0,0,0,1,0,0,0
195,Zambia,0.0,2619.721400,0.000000,135000.0,0,1148.561600,21.700001,0.780800,0.873408,...,23.114420,17.132454,-2.615809,0,0,0,0,0,0,1


### Save edited dataframe as CSV:

In [ ]:
# saving the dataframe
df2.to_csv('inequality_edited.csv')

## Create X and Y vars

In [ ]:
# create set of variables to use / exclude Y

# 1. List of columns to exclude
exclude_columns = ['id', 'country','gini_disp_ext'] # exclude y variable from x's

# 2. Create a list of columns to keep
vars = [col for col in df2.columns if col not in exclude_columns]
print (vars)
# Now you have a list `keep_columns` containing all the columns you want to keep
x = df2.loc[:, vars].values # first param is row, second param is the variables
print (x)
# also create Y while we're at it for use later on in regressions
y = df2.loc[:, 'gini_disp_ext'].values

['federalism_gt', 'gdp', 'statehiste1500_02n', 'origtime2', 'legor_so', 'eleva', 'avg_temp', 'al_ethnic', 'al_language', 'al_religion', 'english_legal_origin', 'arablelandoflandareaag', 'healthexpenditurepublicof', 'maddison_gdppc_1990_estimate_ln', 'duration_global', 'lp_lat_abst_fill', 'mountains', 'log_ocdistance_new', 'pmean', 'irri_impact5', 'frstdays', 'eur_pct_est_smooth', 'sd_emeanclip', 'v2x_corr', 'v2x_polyarchy', 'urbanpopulationoftotalpop', 'dist2suitable_km_new', 'agric_value_added', 'generalgovernmentfinalconsump_ln', 'fixedtelephonesubscriptionsp', 'employmentinagricultureof', 'accesstoelectricityofpopu', 'populationdensitypeoplepers_ln', 'capacity', 'tradeofgdpnetrdgnfszs_ln', 'lsc_ipo', 'slave_descendants_ext', 'pln_sxhr_mean', 'agyears_ext', 'batdeath_pc_cum_ln', 'popd_1500ad', 'popd_1500ad_ln', 'grg_a_lum00pc', 'v1_a_lum00pc', 'eur_pct_homog', 'mal_max', 'tech1500_ext', 'sd_index_ext', 'rugged', 'soil', 'tropical', 'adj_index', 'East Asia and the Pacific', 'Europe an

In [ ]:
df2.describe()

,federalism_gt,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,al_language,al_religion,...,soil,tropical,adj_index,East Asia and the Pacific,Europe and Central Asia,Latin America and the Caribbean,Middle East and North Africa,North America,South Asia,Sub-Saharan Africa
count,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,...,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000,187.000000
mean,0.326203,18310.803353,0.214657,59065.919786,0.176471,586.440638,19.854710,0.438461,0.388880,0.435333,...,38.679897,41.321144,-0.007048,0.101604,0.251337,0.171123,0.144385,0.010695,0.074866,0.245989
std,0.692128,15056.646387,0.204591,45875.756181,0.382243,565.307382,7.868019,0.250959,0.273186,0.233707,...,25.267663,45.570809,1.497763,0.302938,0.434946,0.377627,0.352423,0.103139,0.263882,0.431828
min,0.000000,1333.422200,0.000000,200.000000,0.000000,1.000000,-4.800000,0.000000,0.002113,0.002286,...,0.000000,0.000000,-2.984061,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,0.000000,6747.803429,0.000341,15000.000000,0.000000,217.816215,12.550000,0.207568,0.139191,0.227961,...,17.739778,0.000000,-1.093057,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
50%,0.000000,12679.182646,0.162276,52000.000000,0.000000,415.893010,23.200001,0.429400,0.352571,0.446989,...,39.341293,8.400757,0.162782,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
75%,0.000000,28260.808800,0.377542,75000.000000,0.000000,715.874175,26.100000,0.653700,0.615102,0.639772,...,56.213890,100.000000,1.219547,0.000000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000
max,2.000000,92750.086000,0.758315,160000.000000,1.000000,3362.000000,29.100000,0.930175,0.922679,0.860260,...,100.000000,100.000000,2.695777,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000,1.000000


### Normalize X's

In [ ]:
# normalize x
x_norm = StandardScaler().fit_transform(x) # how do i make belgium, etc not strings? encode them?
x_norm

array([[-0.47257013,  0.66261216,  1.01938707, ..., -0.10397505,
        -0.28447294, -0.57117522],
       [-0.47257013,  0.81615103,  0.78566554, ..., -0.10397505,
        -0.28447294, -0.57117522],
       [-0.47257013,  0.66261216,  0.93394242, ..., -0.10397505,
        -0.28447294, -0.57117522],
       ...,
       [-0.47257013,  0.66261216,  1.84801782, ..., -0.10397505,
        -0.28447294, -0.57117522],
       [-0.47257013, -1.04493427, -1.05201699, ..., -0.10397505,
        -0.28447294,  1.75077623],
       [-0.47257013, -0.77002815, -1.05201699, ..., -0.10397505,
        -0.28447294,  1.75077623]])

### PCA

#### Theory

In [ ]:
# pca agriculture
agr_vars = ['arablelandoflandareaag','pmean','pln_sxhr_mean','soil', 'irri_impact5', 'tropical', 'agric_value_added']
# check if better or worse with agric value added: OG w/o = 0.96151444
agr_group = df2.loc[:, agr_vars].values
pca_agriculture = PCA(n_components=1)

# pca fractionalization
fractionalization_vars = ['al_ethnic','al_language']
fractionalization_group = df2.loc[:, fractionalization_vars].values
pca_fractionalization = PCA(n_components=1)

# try pca -- imagine theory suggests you look for a 1d latent representation for access to ocean
ocean_vars = ['log_ocdistance_new','dist2suitable_km_new']
ocean_group = df2.loc[:, ocean_vars].values
pca_ocean = PCA(n_components=1)

# try pca -- imagine theory suggests you look for a 1d latent representation for access to v-dem
vdem_vars = ['v2x_corr','v2x_polyarchy']
vdem_group = df2.loc[:, vdem_vars].values
pca_vdem = PCA(n_components=1)

# try pca -- imagine theory suggests you look for a 1d latent representation for access to agr skill
agrskill_vars = ['statehiste1500_02n','employmentinagricultureof','agyears_ext']
agrskill_group = df2.loc[:, agrskill_vars].values
pca_agrskill = PCA(n_components=1)

# latent var for terrain
terrain_vars = ['rugged', 'eleva','mountains', 'sd_emeanclip']
terrain_group = df2.loc[:, terrain_vars].values
pca_terrain = PCA(n_components=1)

# OTHER IDEAS
# population
pop_vars = ['urbanpopulationoftotalpop', 'populationdensitypeoplepers_ln', 'popd_1500ad_ln']
pop_group = df2.loc[:, pop_vars].values
pca_pop = PCA(n_components=1)

# pca english pop
eng_vars = ['english_legal_origin', 'eur_pct_est_smooth', 'eur_pct_homog']
eng_group = df2.loc[:, eng_vars].values
pca_eng = PCA(n_components=1)

In [ ]:
# create 1 dimensional representation
latent_var_agr = pca_agriculture.fit_transform(agr_group)
latent_var_fractionalization = pca_fractionalization.fit_transform(fractionalization_group)
latent_var_ocean = pca_ocean.fit_transform(ocean_group)
latent_var_vdem = pca_vdem.fit_transform(vdem_group)
latent_var_agrskill = pca_agrskill.fit_transform(agrskill_group)
latent_var_terrain = pca_terrain.fit_transform(terrain_group)
latent_var_eng = pca_eng.fit_transform(eng_group)
latent_var_pop = pca_pop.fit_transform(pop_group)

# check to see if this is right
print ("Variance explained by each latent variable in PCA: ", pca_agriculture.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_fractionalization.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_ocean.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_vdem.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_agrskill.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_terrain.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_eng.explained_variance_ratio_) # more variance explained the better
print ("Variance explained by each latent variable in PCA: ", pca_pop.explained_variance_ratio_) # more variance explained the better

Variance explained by each latent variable in PCA:  [1.]
Variance explained by each latent variable in PCA:  [0.83849374]
Variance explained by each latent variable in PCA:  [1.]
Variance explained by each latent variable in PCA:  [0.81580359]
Variance explained by each latent variable in PCA:  [0.99991861]
Variance explained by each latent variable in PCA:  [0.99999677]
Variance explained by each latent variable in PCA:  [0.95178089]
Variance explained by each latent variable in PCA:  [0.99449972]


#### Calcs

In [ ]:
correlation_matrix = x = df2.loc[:, vars].corr()
correlation_matrix

,federalism_gt,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,al_language,al_religion,...,soil,tropical,adj_index,East Asia and the Pacific,Europe and Central Asia,Latin America and the Caribbean,Middle East and North Africa,North America,South Asia,Sub-Saharan Africa
federalism_gt,1.000000,0.077637,0.098108,-0.076580,-0.076505,-0.004872,-0.042094,-0.008644,-0.009412,0.079706,...,0.063242,-0.021265,0.166004,0.097493,-0.041640,0.052690,-0.061881,0.252122,0.042188,-0.108026
gdp,0.077637,1.000000,0.296758,-0.358196,-0.006399,-0.148121,-0.418826,-0.308865,-0.293820,-0.175314,...,-0.111291,-0.492695,0.492687,-0.051726,0.472885,-0.170404,0.272196,0.226500,-0.208512,-0.439821
statehiste1500_02n,0.098108,0.296758,1.000000,-0.086375,0.110101,0.157500,-0.133531,-0.181668,-0.145649,-0.320929,...,-0.091925,-0.487673,0.606549,-0.002365,0.175694,-0.339546,0.559367,-0.109384,0.174382,-0.415322
origtime2,-0.076580,-0.358196,-0.086375,1.000000,-0.203351,0.151511,0.400403,0.489816,0.605017,0.234029,...,-0.190187,0.128731,-0.515751,-0.013753,-0.326875,-0.350771,-0.015112,-0.084234,-0.017915,0.689026
legor_so,-0.076505,-0.006399,0.110101,-0.203351,1.000000,0.087410,-0.501802,-0.119714,-0.079221,0.023904,...,0.116745,-0.309178,0.233423,-0.016387,0.540233,-0.173086,-0.150250,-0.048131,0.028218,-0.264403
eleva,-0.004872,-0.148121,0.157500,0.151511,0.087410,1.000000,-0.299939,0.120691,0.129824,-0.042610,...,-0.117988,-0.283439,-0.018651,-0.014223,-0.042310,-0.122448,0.106751,0.002875,0.010256,0.065596
avg_temp,-0.042094,-0.418826,-0.133531,0.400403,-0.501802,-0.299939,1.000000,0.319653,0.215169,0.037047,...,-0.073898,0.660257,-0.398090,0.096677,-0.723888,0.257581,0.103448,-0.231944,0.123606,0.331482
al_ethnic,-0.008644,-0.308865,-0.181668,0.489816,-0.119714,0.120691,0.319653,1.000000,0.674093,0.146253,...,-0.261923,0.191319,-0.372834,-0.148770,-0.319608,-0.048820,0.019220,0.067633,-0.079916,0.485969
al_language,-0.009412,-0.293820,-0.145649,0.605017,-0.079221,0.129824,0.215169,0.674093,1.000000,0.259698,...,-0.114660,0.146818,-0.408181,0.018398,-0.186951,-0.344611,-0.089972,0.009692,0.080344,0.498767
al_religion,0.079706,-0.175314,-0.320929,0.234029,0.023904,-0.042610,0.037047,0.146253,0.259698,1.000000,...,0.193884,0.236784,-0.246521,0.127492,-0.066374,0.006871,-0.271820,0.144796,-0.026392,0.174787


In [ ]:
from itertools import combinations

# Set a threshold for high correlation
threshold = 0.7  # You can adjust this threshold as needed
group_size = 4   # Set the desired group size (e.g., 3 for triplets)

# Initialize a list to hold the correlated variable groups
high_correlation_groups = []

# Loop through combinations of variables of the specified size
for group in combinations(correlation_matrix.columns, group_size):
    # Check if all pairs within the group have high correlation
    all_high_corr = True
    for i in range(len(group)):
        for j in range(i + 1, len(group)):
            if abs(correlation_matrix.loc[group[i], group[j]]) <= threshold:
                all_high_corr = False
                break
        if not all_high_corr:
            break
  # If all pairs have high correlation, calculate variance explained using PCA
    if all_high_corr:
        # Create a sub-dataframe with the selected variables
        sub_df = df2[list(group)]

        # Apply PCA to the sub-dataframe
        pca = PCA(n_components=1)
        pca.fit(sub_df)

        # Get the variance explained by the first principal component
        variance_explained = pca.explained_variance_ratio_[0]

        # Append the group and its variance explained to the list
        high_correlation_groups.append((*group, variance_explained))

# Convert the list to a DataFrame for better visualization if needed
high_corr_df = pd.DataFrame(high_correlation_groups, columns=[f'Variable {i+1}' for i in range(group_size)] + ['Variance Explained'])

# Display the high correlation groups and their variance explained
high_corr_df

,Variable 1,Variable 2,Variable 3,Variable 4,Variance Explained
0,avg_temp,lp_lat_abst_fill,frstdays,Europe and Central Asia,0.94596


In [ ]:
# fit pca with reccomended variables
selected_vars = ['avg_temp','lp_lat_abst_fill',	'frstdays', 'Europe and Central Asia']
selected_group = df2.loc[:, selected_vars].values

pca_final = PCA(n_components=1)  # Use 1 if you want one latent component
latent_variable = pca_final.fit_transform(selected_group)

# Add the latent variable to the DataFrame
df2['latent_variable'] = latent_variable[:,0]

### Add PCA to df

**Explanation of PCA and the latent_var_xyz variables:**


*latent_var_xyz* holds the results of the PCA transformation applied to the [] variables in my dataset. After performing PCA, latent_var_xyz should be a NumPy array where each column corresponds to a principal component, and each row corresponds to an observation (sample) in df2.

The first principal component (PC1) explains the largest amount of variance, the second principal component (PC2) explains the second largest amount, and so on. If I performed PCA with n_components=k, latent_var_agr will contain k columns, with each column representing one principal component.

In [ ]:
# create new dataframe with the latent variables from pca1 (add more latent vars)
df2['pca_agr'] = latent_var_agr[:,0] # aka take all rows from col 0 of latent_var_agr.
df2['pca_fractionalization'] = latent_var_fractionalization[:,0]
df2['pca_ocean'] = latent_var_ocean[:,0]
df2['pca_vdem'] = latent_var_vdem[:,0]
df2['pca_agrskill'] = latent_var_agrskill[:,0]
df2['pca_terrain'] = latent_var_terrain[:,0]
df2['pca_eng'] = latent_var_eng[:,0]
df2['pca_pop'] = latent_var_pop[:,0]

# add the latent variables to x_norm
x_norm = np.append(x_norm,latent_var_agr,1)
x_norm = np.append(x_norm,latent_var_fractionalization,1)
x_norm = np.append(x_norm,latent_var_ocean,1)
x_norm = np.append(x_norm,latent_var_vdem,1)
x_norm = np.append(x_norm,latent_var_agrskill,1)
x_norm = np.append(x_norm,latent_var_terrain,1)
x_norm = np.append(x_norm,latent_var_eng,1)
x_norm = np.append(x_norm,latent_var_pop,1)

note: pca's are not standardized!!

In [ ]:
df2

,country,federalism_gt,gdp,statehiste1500_02n,origtime2,legor_so,eleva,avg_temp,al_ethnic,al_language,...,Sub-Saharan Africa,latent_variable,pca_agr,pca_fractionalization,pca_ocean,pca_vdem,pca_agrskill,pca_terrain,pca_eng,pca_pop
0,Afghanistan,0.0,28260.808800,0.422657,40000.0,0,1836.798300,13.800000,0.769345,0.614146,...,0,14.286995,-2.947387e+14,0.387806,561038.911633,0.392085,4494.655918,1250.358361,-23.837821,-32.244772
1,Albania,0.0,30566.399796,0.374967,45000.0,1,725.777650,12.500000,0.220426,0.039925,...,0,9.180638,-1.589878e+15,-0.405934,-288051.453383,0.174058,2994.686624,139.339807,69.108631,-6.452945
2,Algeria,0.0,28260.808800,0.405222,40000.0,0,585.317870,23.400000,0.339400,0.442662,...,0,-5.023769,4.265394e+15,-0.025183,455417.851631,0.281782,-505.249622,-1.123964,-22.871462,9.986879
3,Angola,0.0,6747.803429,0.050186,135000.0,0,1099.466300,21.600000,0.786720,0.787019,...,1,-6.623340,1.937038e+15,0.528957,165453.411632,0.504394,-3255.295515,513.024438,-22.488599,2.163484
4,Antigua,1.0,12679.182646,0.062178,58917.0,0,60.476498,26.500000,0.164256,0.106328,...,0,-9.785850,-1.699349e+15,-0.393282,-343101.521435,-0.051985,-3005.249772,-525.965458,-21.510187,-27.352052
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
192,Venezuela,2.0,19012.992000,0.000000,15000.0,0,374.298950,25.900000,0.496600,0.068580,...,0,-10.325019,5.082995e+14,-0.201776,-71864.308370,0.358712,-705.236269,-212.142408,20.239591,32.646426
193,Vietnam,0.0,7304.192150,0.474578,75000.0,1,420.906520,24.799999,0.238308,0.237708,...,0,-9.736123,-1.439730e+15,-0.245765,-264268.250382,0.323233,1494.682239,-165.532929,-23.836688,-26.848579
194,Yemen,0.0,28260.808800,0.591734,85000.0,0,999.000000,24.700001,0.450171,0.007982,...,0,-2.924990,6.550177e+15,-0.277935,-174020.018374,0.417329,3094.720612,412.560355,-23.837821,-25.306386
195,Zambia,0.0,2619.721400,0.000000,135000.0,0,1148.561600,21.700001,0.780800,0.873408,...,1,-6.425604,-2.094680e+15,0.589835,631220.911632,-0.220187,-2705.353085,562.119166,-23.072567,-17.479980


### Train Models


In [ ]:
# 1. List of columns to exclude
exclude_columns = ['id', 'country','gini_disp_ext','arablelandoflandareaag','pmean','pln_sxhr_mean','soil', 'irri_impact5', 'frstdays',
                   'tropical', 'avg_temp', 'agric_value_added', 'al_ethnic','al_language','log_ocdistance_new','dist2suitable_km_new','v2x_corr','v2x_polyarchy', 'statehiste1500_02n',
                   'employmentinagricultureof','agyears_ext', 'rugged', 'eleva','mountains', 'sd_emeanclip', 'english_legal_origin', 'eur_pct_est_smooth', 'eur_pct_homog', 'pca_eng',
                   'urbanpopulationoftotalpop', 'populationdensitypeoplepers_ln', 'popd_1500ad_ln', 'popd_1500ad', 'latent_variable',
                   'gdp', 'taxrevenueofgdpgctaxt_2', 'generalgovernmentfinalconsump_ln', 'slave_descendants_ext', 'batdeath_pc_cum_ln',
                   'fixedtelephonesubscriptionsp',
                   'legor_so', 'healthexpenditurepublicof'
                   'Middle East and North Africa', 'South Asia',
                   'tradeofgdpnetrdgnfszs_ln', 'capacity', 'grg_a_lum00pc', 'pca_agrskill',
                   'origtime2',
                   'duration_global']# best everything. last line is newly added, exclude y variable from x's

# 2. Create a list of columns to keep
IVs = [col for col in df2.columns if col not in exclude_columns]

# create train / test split using dataframe
x_train, x_test, y_train, y_test = train_test_split(df2.loc[:, IVs], df2.loc[:, 'gini_disp_ext'], test_size=0.25, random_state=13)

# make sure results make sense
print (x_train.shape, y_train.shape)
print (x_test.shape, y_test.shape)
IVs

(140, 24) (140,)
(47, 24) (47,)


['federalism_gt',
 'al_religion',
 'healthexpenditurepublicof',
 'maddison_gdppc_1990_estimate_ln',
 'lp_lat_abst_fill',
 'accesstoelectricityofpopu',
 'lsc_ipo',
 'v1_a_lum00pc',
 'mal_max',
 'tech1500_ext',
 'sd_index_ext',
 'adj_index',
 'East Asia and the Pacific',
 'Europe and Central Asia',
 'Latin America and the Caribbean',
 'Middle East and North Africa',
 'North America',
 'Sub-Saharan Africa',
 'pca_agr',
 'pca_fractionalization',
 'pca_ocean',
 'pca_vdem',
 'pca_terrain',
 'pca_pop']

In [ ]:
# try both linear and polynomial of different degrees
linear_model = LinearRegression() # OG = linear_model = LinearRegression(normalize=True)
p2_model = LinearRegression()
p3_model = LinearRegression()

# create polynomial features
p2_features = PolynomialFeatures(degree=2)
p2_train = p2_features.fit_transform(x_train)
p2_test = p2_features.fit_transform(x_test)

p3_features = PolynomialFeatures(degree=3)
p3_train = p3_features.fit_transform(x_train)
p3_test = p3_features.fit_transform(x_test)

In [ ]:
# now do estimation of models
lin_1 = linear_model.fit(x_train, y_train)
p2_1 = p2_model.fit(p2_train, y_train)
p3_1 = p3_model.fit(p3_train, y_train)

# predict values for test sets
lin1_predict = lin_1.predict(x_test)
p2_predict = p2_1.predict(p2_test)
p3_predict = p3_1.predict(p3_test)

print(mean_squared_error(lin1_predict, y_test))
print(mean_squared_error(p2_predict, y_test))
print(mean_squared_error(p3_predict, y_test))

# linreg MSE: 14.509696934370048 third last line
# MSE 13.504100650910788 secondlast line

13.504100650910788
70.94873355002441
48.27533988270652


Get linear equation

In [ ]:
# Extract coefficients and intercept
coefficients = lin_1.coef_
intercept = lin_1.intercept_

# Construct the regression equation
equation = f"y = {intercept:.4f}"
for coef, name in zip(coefficients,x_train.columns):
  if coef != 0:
    equation += f" + {coef:.4f} * {name}"

# Output the equation
print("Linear Regression Equation:", equation)

Linear Regression Equation: y = 49.7147 + 1.0230 * federalism_gt + 0.1745 * al_religion + -0.1626 * healthexpenditurepublicof + -1.1320 * maddison_gdppc_1990_estimate_ln + -0.2555 * lp_lat_abst_fill + 0.0454 * accesstoelectricityofpopu + -0.1193 * lsc_ipo + 0.1179 * v1_a_lum00pc + 0.6549 * mal_max + -0.5941 * tech1500_ext + 1.6976 * sd_index_ext + -2.5284 * adj_index + 0.1041 * East Asia and the Pacific + -0.4244 * Europe and Central Asia + 0.5049 * Latin America and the Caribbean + -0.4889 * Middle East and North Africa + 0.2829 * Sub-Saharan Africa + -0.0000 * pca_agr + 0.2881 * pca_fractionalization + -0.0000 * pca_ocean + 0.2535 * pca_vdem + 0.0032 * pca_terrain + 0.0206 * pca_pop


# Regularize

### Try Lasso

In [ ]:
# use k-fold with regularization
# remember: alpha here is lambda in most treatments
from sklearn.linear_model import LassoCV

#Lasso Cross validation
lasso_1 = LassoCV(alphas = [0.0001, 0.001,0.01, 0.1, 1, 10], random_state=0).fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1.9832461577843787, tolerance: 0.8272633804076086
  model = cd_fast.enet_coordinate_descent_gram(


In [ ]:
selected_features = x_train.columns[lasso_1.coef_ != 0] # before: selected_features = x_train.columns[lasso_1.coef_ != 0]
print("Selected features via Lasso:", selected_features)

Selected features via Lasso: Index(['federalism_gt', 'healthexpenditurepublicof',
       'maddison_gdppc_1990_estimate_ln', 'accesstoelectricityofpopu',
       'lsc_ipo', 'v1_a_lum00pc', 'sd_index_ext', 'adj_index',
       'Europe and Central Asia', 'Sub-Saharan Africa', 'pca_agr', 'pca_ocean',
       'pca_terrain', 'pca_pop'],
      dtype='object')


In [ ]:
# Optimal alpha value selected by cross-validation
print("Optimal alpha:", lasso_1.alpha_)

# Coefficients of the features
print("Lasso coefficients:", lasso_1.coef_)

Optimal alpha: 0.1
Lasso coefficients: [ 5.51849423e-01  0.00000000e+00 -3.44624899e-02 -3.91817848e-01
 -0.00000000e+00  4.08439951e-02 -4.85647737e-02  1.95325967e+00
  0.00000000e+00 -0.00000000e+00  1.52256348e+00 -2.23556191e+00
  0.00000000e+00 -5.16612132e+00 -0.00000000e+00 -0.00000000e+00
  0.00000000e+00  2.56030606e+00 -2.57370360e-17  0.00000000e+00
 -2.69891292e-06 -0.00000000e+00  2.84604232e-03 -9.43685222e-03]


Lasso r^2 values

In [ ]:
# results in r^2
print(lasso_1.score(x_train, y_train))
print(lasso_1.score(x_test, y_test))

0.6441709463486007
0.6558125094381687


Lasso MSEs

In [ ]:
from sklearn.metrics import mean_squared_error

# Predict on test set
y_train_pred = lasso_1.predict(x_train)
y_test_pred = lasso_1.predict(x_test)

# Calculate MSE
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("Training MSE:", train_mse)
print("Test MSE:", test_mse)

# Training MSE: 25.153862797189124
# Test MSE: 17.706820192974654 second last line

Training MSE: 25.153862797189124
Test MSE: 17.706820192974654


In [ ]:
# Extract the intercept and coefficients
intercept = lasso_1.intercept_
coefficients = lasso_1.coef_

# Display the final regression equation
equation = f"y = {intercept:.3f} "
for coef, feature in zip(coefficients, IVs):
    if coef != 0.000:  # Include only features with non-zero coefficients
        equation += f"+ ({coef:.3f}) * {feature} "

print("Final regression equation from Lasso:", equation)

Final regression equation from Lasso: y = 41.436 + (0.552) * federalism_gt + (-0.034) * healthexpenditurepublicof + (-0.392) * maddison_gdppc_1990_estimate_ln + (0.041) * accesstoelectricityofpopu + (-0.049) * lsc_ipo + (1.953) * v1_a_lum00pc + (1.523) * sd_index_ext + (-2.236) * adj_index + (-5.166) * Europe and Central Asia + (2.560) * Sub-Saharan Africa + (-0.000) * pca_agr + (-0.000) * pca_ocean + (0.003) * pca_terrain + (-0.009) * pca_pop 


### Try Elastic Net

In [ ]:
from sklearn.linear_model import ElasticNetCV

# Try ElasticNetCV with cross-validation
elastic_net_cv = ElasticNetCV(
    alphas=np.logspace(-6, 0, 100),
    l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9],
    cv=5,
    random_state=0,
    max_iter=10000
)

In [ ]:
elastic_net_cv.fit(x_train, y_train)

/usr/local/lib/python3.10/dist-packages/sklearn/linear_model/_coordinate_descent.py:683: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 121.12736933432734, tolerance: 0.8867034195609835
  model = cd_fast.enet_coordinate_descent_gram(


ElasticNetCV(alphas=array([1.00000000e-06, 1.14975700e-06, 1.32194115e-06, 1.51991108e-06,
       1.74752840e-06, 2.00923300e-06, 2.31012970e-06, 2.65608778e-06,
       3.05385551e-06, 3.51119173e-06, 4.03701726e-06, 4.64158883e-06,
       5.33669923e-06, 6.13590727e-06, 7.05480231e-06, 8.11130831e-06,
       9.32603347e-06, 1.07226722e-05, 1.23284674e-05, 1.41747416e-05,
       1.62975083e-05, 1.873817...
       7.05480231e-02, 8.11130831e-02, 9.32603347e-02, 1.07226722e-01,
       1.23284674e-01, 1.41747416e-01, 1.62975083e-01, 1.87381742e-01,
       2.15443469e-01, 2.47707636e-01, 2.84803587e-01, 3.27454916e-01,
       3.76493581e-01, 4.32876128e-01, 4.97702356e-01, 5.72236766e-01,
       6.57933225e-01, 7.56463328e-01, 8.69749003e-01, 1.00000000e+00]),
             cv=5, l1_ratio=[0.1, 0.3, 0.5, 0.7, 0.9], max_iter=10000,
             random_state=0)

In [ ]:
print("X_train shape:", x_train.shape)  # Should show (number of samples, number of features)
print("ElasticNet coefficients shape:", elastic_net_cv.coef_.shape)  # Should match the number of features in X_train

X_train shape: (140, 24)
ElasticNet coefficients shape: (24,)


Elastic Net r^2

In [ ]:
# Get R² score on training data
train_score = elastic_net_cv.score(x_train, y_train)
print("Training R² score:", train_score)

# Get R² score on test data
test_score = elastic_net_cv.score(x_test, y_test)
print("Test R² score:", test_score)

Training R² score: 0.6345101877378165
Test R² score: 0.6864772887380828


Elastic Net MSEs

In [ ]:
from sklearn.metrics import mean_squared_error

# Predict on test set
y_train_pred = elastic_net_cv.predict(x_train)
y_test_pred = elastic_net_cv.predict(x_test)

# Calculate MSE
train_mse = mean_squared_error(y_train, y_train_pred)
test_mse = mean_squared_error(y_test, y_test_pred)

print("Training MSE:", train_mse)
print("Test MSE:", test_mse)

#best: all until last line
#Training MSE: 25.836790158287922
# Test MSE: 16.129262181105865

Training MSE: 25.836790158287922
Test MSE: 16.129262181105865


Elastic Net Equation

In [ ]:
# Print coefficients and intercept
intercept = elastic_net_cv.intercept_
coefficients = elastic_net_cv.coef_

# Display the final regression equation
equation = f"y = {intercept:.3f} "
for coef, feature in zip(coefficients, IVs):
    if coef != 0:
        equation += f"+ ({coef:.3f}) * {feature} "

print("Final regression equation from ElasticNet:", equation)

Final regression equation from ElasticNet: y = 43.589 + (0.523) * federalism_gt + (-0.065) * healthexpenditurepublicof + (-0.435) * maddison_gdppc_1990_estimate_ln + (0.030) * accesstoelectricityofpopu + (-0.068) * lsc_ipo + (0.953) * v1_a_lum00pc + (1.568) * sd_index_ext + (-2.299) * adj_index + (-4.211) * Europe and Central Asia + (1.744) * Sub-Saharan Africa + (-0.000) * pca_agr + (-0.000) * pca_ocean + (0.003) * pca_terrain + (-0.003) * pca_pop 
